# PySpark Tutorial - Dataframes
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

Up to now, we've see the RDD interface to PySpark. The RDD is a building block for more capable data structures such as the **dataframe** and **database**. These data structures are part of the [PySpark SQL library](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html) which, as the name implies, is influenced by standard SQL practices and queries.

The PySpark library has the **dataframe API**, but it does not support the **database API** -- that's only accessible via the Scala and Java libraries and through SQL queries.

The **database** is effectively an SQL relation -- i.e. rows and columns with a specific schema. The **dataframe** takes a little futher and constructs a labeled dataframe similar to the [Python Pandas](https://pandas.pydata.org/) interface or the [R dplyr](https://cran.r-project.org/web/packages/dplyr/vignettes/dplyr.html) interface for R.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import operator

We're going to use an airline information database as the example. You can download extended versions of the database [at this Dept. of Transportation website](https://www.transtats.bts.gov/DL_SelectFields.asp), but the data we're using is distributed with the course notes.

As with the RDD interface, we need a "context" to a remote machine. The [Spark SQL tutorial](https://spark.apache.org/docs/latest/sql-getting-started.html) has some information on this, but for complete information you need to look at the [Spark API documentation.](https://spark.apache.org/docs/latest/api/python/)

In this example, we're creating a location session (i.e. CPU's on JupyterHub).

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master("local[*]")\
    .getOrCreate()

There are many ways to load data, including HDFS, a format called [Parquet](http://parquet.apache.org/), CSV files and so on. We'll use a compressed CSV file of the airline data.

In [3]:
flights = spark.read.load('airline-ontime-reporting.csv.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

The dataframe has a **schema** or type for each entry. All entries must have the same type or we'll see operations fail.

In [4]:
flights.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
 |-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
 |-- DEST_CITY_MARKET_ID: integer (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- ARR_TIME: integer (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- 

In [6]:
flights.columns

['YEAR',
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'OP_UNIQUE_CARRIER',
 'ORIGIN_AIRPORT_ID',
 'ORIGIN_AIRPORT_SEQ_ID',
 'ORIGIN_CITY_MARKET_ID',
 'ORIGIN',
 'ORIGIN_CITY_NAME',
 'DEST_AIRPORT_ID',
 'DEST_AIRPORT_SEQ_ID',
 'DEST_CITY_MARKET_ID',
 'DEST',
 'DEST_CITY_NAME',
 'CRS_DEP_TIME',
 'DEP_DELAY',
 'ARR_TIME',
 'ARR_DELAY',
 'CANCELLED',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'LATE_AIRCRAFT_DELAY',
 '_c23']

In [7]:
print("There are", len(flights.columns), "columns and ", flights.count(), "rows")

There are 24 columns and  583985 rows


In [8]:
flights.dtypes

[('YEAR', 'int'),
 ('MONTH', 'int'),
 ('DAY_OF_MONTH', 'int'),
 ('DAY_OF_WEEK', 'int'),
 ('OP_UNIQUE_CARRIER', 'string'),
 ('ORIGIN_AIRPORT_ID', 'int'),
 ('ORIGIN_AIRPORT_SEQ_ID', 'int'),
 ('ORIGIN_CITY_MARKET_ID', 'int'),
 ('ORIGIN', 'string'),
 ('ORIGIN_CITY_NAME', 'string'),
 ('DEST_AIRPORT_ID', 'int'),
 ('DEST_AIRPORT_SEQ_ID', 'int'),
 ('DEST_CITY_MARKET_ID', 'int'),
 ('DEST', 'string'),
 ('DEST_CITY_NAME', 'string'),
 ('CRS_DEP_TIME', 'int'),
 ('DEP_DELAY', 'double'),
 ('ARR_TIME', 'int'),
 ('ARR_DELAY', 'double'),
 ('CANCELLED', 'double'),
 ('CARRIER_DELAY', 'double'),
 ('WEATHER_DELAY', 'double'),
 ('LATE_AIRCRAFT_DELAY', 'double'),
 ('_c23', 'string')]

The schema is inferred, but it can also be defined explicitly.

Note that one column is labeled `_c23`, which is showing up as "null". Perhaps this is bad data import?

Lets look at some of the values.

In [9]:
flights.show(5, truncate=False)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|_c23|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+
|2019|1    |19          |6     

Let's pull out the values in one column -- the `select` method can be used to produce a new dataframe with just that column as an entry.

In [11]:
flights.select('_c23').show(5)

+----+
|_c23|
+----+
|null|
|null|
|null|
|null|
|null|
+----+
only showing top 5 rows



And we can slice out multiple columns, similar to Pandas. Again, this produces a new dataframe.

In [12]:
flights.select(['year', '_c23']).show(5)

+----+----+
|year|_c23|
+----+----+
|2019|null|
|2019|null|
|2019|null|
|2019|null|
|2019|null|
+----+----+
only showing top 5 rows



Alternatively, we can produce a
[Column object which has its own methods](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=column#pyspark.sql.Column). These are typically used in **column expressions** that produce indicies that can be used when selecting or filtering data.

For example, let's find all the rows where the mystery `_c23` column is not null.

In [13]:
flights.filter( flights._c23.isNotNull()).show(5)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|_c23|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+
+----+-----+------------+------

Hmm.... This liooks like all the values are null. We could confirm this by selecting the column and looking at the distinct elements.

In [14]:
flights.select('_c23').distinct().show()

+----+
|_c23|
+----+
|null|
+----+



This this column is null, lets just drop it.

In [15]:
newFlights = flights.drop('_c23')

In [16]:
newFlights.show(5)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+
|2019|    1|          19|          6|         

We often work with multiple columns of data in a dataframe. Some methods just use column names (corr, cov, crosstab, describe) and others can use column references, such as `newAir.ORIGIN`.

There are also a number of methods that work on columns or column expressions -- we've been using `select` already.

* `cube(*cols)`: column names (string) or column expressions or **both**.
* `drop(*cols)`: ***a list of column names OR a single column expression.***
* `groupBy(*cols)`: column name (string) or column expression or **both**.
* `rollup(*cols)`: column name (string) or column expression or **both**.
* `select(*cols)`: column name (string) or column expression or **both**.
* `sort(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sortWithinPartitions(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `orderBy(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sampleBy(col, fractions, sed=None)`: a column name.
* `toDF(*cols)`: **a list of column names (string).**
* `withColumn(colName, col)`: `colName` refers to column name; `col` refers to a column expression.
* `withColumnRenamed(existing, new)`: takes column names as arguments.
* `filter(condition)`: ***condition** refers to a column expression that returns `types.BooleanType` of values. 

In [17]:
newFlights.groupBy(newFlights.ORIGIN).count().collect()

[Row(ORIGIN='BGM', count=61),
 Row(ORIGIN='PSE', count=65),
 Row(ORIGIN='INL', count=53),
 Row(ORIGIN='MSY', count=4597),
 Row(ORIGIN='PPG', count=11),
 Row(ORIGIN='GEG', count=934),
 Row(ORIGIN='DRT', count=61),
 Row(ORIGIN='SNA', count=3416),
 Row(ORIGIN='BUR', count=2377),
 Row(ORIGIN='GRB', count=357),
 Row(ORIGIN='GTF', count=117),
 Row(ORIGIN='IDA', count=134),
 Row(ORIGIN='GRR', count=1506),
 Row(ORIGIN='LWB', count=52),
 Row(ORIGIN='PVU', count=46),
 Row(ORIGIN='JLN', count=62),
 Row(ORIGIN='EUG', count=313),
 Row(ORIGIN='PSG', count=62),
 Row(ORIGIN='GSO', count=1233),
 Row(ORIGIN='PVD', count=1355),
 Row(ORIGIN='MYR', count=571),
 Row(ORIGIN='OAK', count=4186),
 Row(ORIGIN='EAR', count=53),
 Row(ORIGIN='MSN', count=1027),
 Row(ORIGIN='FSM', count=167),
 Row(ORIGIN='FAR', count=431),
 Row(ORIGIN='MQT', count=93),
 Row(ORIGIN='COD', count=60),
 Row(ORIGIN='BTM', count=58),
 Row(ORIGIN='SCC', count=85),
 Row(ORIGIN='ESC', count=54),
 Row(ORIGIN='DCA', count=11839),
 Row(ORIGIN='

In [19]:
newFlights.filter(newFlights.ORIGIN == 'DEN' ).show(5)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+---------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST| DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+---------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+
|2019|    1|           1|          2|      

## Doing Joins

Again, everything boils down to a join in "big data". We can do joins between two dataframes much as in Pandas. Let's load a second dataframe that contains airline identifirs.

In [20]:
airlines = spark.read.load('unique-carriers.csv.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [21]:
airlines.show(5)

+----+--------------------+
|Code|         Description|
+----+--------------------+
| 02Q|       Titan Airways|
| 04Q|  Tradewind Aviation|
| 05Q| Comlux Aviation, AG|
| 06Q|Master Top Linhas...|
| 07Q| Flair Airlines Ltd.|
+----+--------------------+
only showing top 5 rows



In [22]:
flights.select('OP_UNIQUE_CARRIER').show(5)

+-----------------+
|OP_UNIQUE_CARRIER|
+-----------------+
|               9E|
|               9E|
|               9E|
|               9E|
|               9E|
+-----------------+
only showing top 5 rows



In [23]:
flights.join(airlines, airlines.Code == flights.OP_UNIQUE_CARRIER).show(5)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+----+-----------------+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|_c23|Code|      Description|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-----------

## Escape back into the world of RDD's

A dataframe is composed of `Row` objects and a dataframe (and database) is just a collection of those rows. You can pull out the row objects as RDD's and then operate on those, much as we did before.

In [24]:
flights.rdd.filter(lambda x: x['DEST'] == 'DEN').take(5)

[Row(YEAR=2019, MONTH=1, DAY_OF_MONTH=1, DAY_OF_WEEK=2, OP_UNIQUE_CARRIER='AA', ORIGIN_AIRPORT_ID=12892, ORIGIN_AIRPORT_SEQ_ID=1289208, ORIGIN_CITY_MARKET_ID=32575, ORIGIN='LAX', ORIGIN_CITY_NAME='Los Angeles, CA', DEST_AIRPORT_ID=11292, DEST_AIRPORT_SEQ_ID=1129202, DEST_CITY_MARKET_ID=30325, DEST='DEN', DEST_CITY_NAME='Denver, CO', CRS_DEP_TIME=1540, DEP_DELAY=-4.0, ARR_TIME=1915, ARR_DELAY=15.0, CANCELLED=0.0, CARRIER_DELAY=0.0, WEATHER_DELAY=0.0, LATE_AIRCRAFT_DELAY=0.0, _c23=None),
 Row(YEAR=2019, MONTH=1, DAY_OF_MONTH=2, DAY_OF_WEEK=3, OP_UNIQUE_CARRIER='AA', ORIGIN_AIRPORT_ID=12892, ORIGIN_AIRPORT_SEQ_ID=1289208, ORIGIN_CITY_MARKET_ID=32575, ORIGIN='LAX', ORIGIN_CITY_NAME='Los Angeles, CA', DEST_AIRPORT_ID=11292, DEST_AIRPORT_SEQ_ID=1129202, DEST_CITY_MARKET_ID=30325, DEST='DEN', DEST_CITY_NAME='Denver, CO', CRS_DEP_TIME=1540, DEP_DELAY=-3.0, ARR_TIME=1908, ARR_DELAY=8.0, CANCELLED=0.0, CARRIER_DELAY=None, WEATHER_DELAY=None, LATE_AIRCRAFT_DELAY=None, _c23=None),
 Row(YEAR=2019, 

In [25]:
onlyDen = spark.createDataFrame(flights.rdd.filter(lambda x: x['DEST'] == 'DEN'))

ValueError: Some of types cannot be determined by the first 100 rows, please try again with sampling

In [26]:
onlyDen = spark.createDataFrame(flights.rdd.filter(lambda x: x['DEST'] == 'DEN'), 
                                samplingRatio=0.5)

## Using SQL

In [27]:
from pyspark.sql import SQLContext

sqlContext = SQLContext( spark.sparkContext )

In [28]:
sqlContext.registerDataFrameAsTable(onlyDen, "onlyDen")

In [29]:
sqlContext.registerDataFrameAsTable(flights, "flights")

In [30]:
sqlContext.sql("SELECT COUNT(*) from onlyDEN").show(5)

+--------+
|count(1)|
+--------+
|   18498|
+--------+



In [31]:
sqlContext.sql("SELECT COUNT(*) from flights WHERE DEST='DEN'").show(5)

+--------+
|count(1)|
+--------+
|   18498|
+--------+

